# Treinamento do Modelo de Matching

Este notebook demonstra um exemplo de treinamento de um modelo simples de
"match" entre candidatos e vagas. Os dados sintéticos estão em `data_source/`.
Utilizamos atributos do candidato **e** algumas informações da vaga para
calcular a compatibilidade.

In [42]:
import json
from pathlib import Path

import joblib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import sys, os
sys.path.append(os.path.abspath('..'))
from src.data_loader import parse_remuneracao

DATA_DIR = Path('..') / 'data_source'
MODEL_PATH = Path('..') / 'model.joblib'

## 1. Carregamento dos Dados

In [43]:
applicants_file = DATA_DIR / 'applicants.json'
prospects_file = DATA_DIR / 'prospects.json'
jobs_file = DATA_DIR / 'vagas.json'

with open(applicants_file, 'r', encoding='utf-8') as f:
    applicants_raw = json.load(f)

with open(prospects_file, 'r', encoding='utf-8') as f:
    prospects_raw = json.load(f)

with open(jobs_file, 'r', encoding='utf-8') as f:
    jobs_raw = json.load(f)

In [44]:
# Extrai algumas informações básicas dos candidatos
feats = []
for code, data in applicants_raw.items():
    basic = data.get('infos_basicas', {})
    prof = data.get('informacoes_profissionais', {}) or {}

    objective = basic.get('objetivo_profissional') or ''
    title = prof.get('titulo_profissional') or ''
    remun_raw = (prof.get('remuneracao') or '').strip()
    remun_val = parse_remuneracao(remun_raw)

    feats.append({
        'codigo': str(code),
        'objective_len': len(objective),
        'title_len': len(title),
        'remuneracao': remun_val,
    })

feat_df = pd.DataFrame(feats)

In [45]:
# Constrói rótulos e mapeia cada candidato para a vaga associada (quando há)
labels = {}
job_map = {}
for job_id, job in prospects_raw.items():
    for cand in job.get('prospects', []):
        code = str(cand.get('codigo'))
        status = cand.get('situacao_candidado', '')
        hired = 'Contratado' in status
        labels[code] = labels.get(code, False) or hired
        if code not in job_map or hired:
            job_map[code] = str(job_id)

label_df = pd.DataFrame({'codigo': list(labels.keys()),
                         'match': [int(v) for v in labels.values()]})

In [46]:
# Extrai informações das vagas usadas como features
job_feats = []
for job_id, info in jobs_raw.items():
    perfil = info.get('perfil_vaga', {})
    basics = info.get('informacoes_basicas', {})
    job_feats.append({
        'job_id': str(job_id),
        'job_title_len': len(basics.get('titulo_vaga', '') or ''),
        'job_level': perfil.get('nivel profissional', ''),
        'job_english': perfil.get('nivel_ingles', ''),
        'job_area_len': len(perfil.get('areas_atuacao', '') or '')
    })

jobs_df = pd.DataFrame(job_feats)
level_map = {'Júnior': 1, 'Pleno': 2, 'Sênior': 3, 'Especialista': 4}
english_map = {'Básico': 1, 'Intermediário': 2, 'Avançado': 3, 'Fluente': 4}
jobs_df['job_level'] = jobs_df['job_level'].map(level_map).fillna(0)
jobs_df['job_english'] = jobs_df['job_english'].map(english_map).fillna(0)

In [47]:
# Combina todas as informações em um único dataframe
feat_df['job_id'] = feat_df['codigo'].map(job_map)
df = feat_df.merge(label_df, on='codigo', how='inner')
df = df.merge(jobs_df, on='job_id', how='left')
df = df.fillna(0)

# Engenharia de algumas features simples
df['objective_len_sq'] = df['objective_len'] ** 2
df['title_remuneracao'] = df['title_len'] * df['remuneracao']
df['title_job_diff'] = (df['title_len'] - df['job_title_len']).abs()

X = df.drop(columns=['match', 'codigo', 'job_id'])
y = df['match']

## 2. Treinamento do Modelo

In [48]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=100))
])

pipeline.fit(X_train, y_train)
acc = pipeline.score(X_val, y_val)
print(f'Acurácia de validação: {acc:.2f}')

Acurácia de validação: 0.90


## 3. Salvando o Modelo

In [49]:
joblib.dump(pipeline, MODEL_PATH)
print('Modelo salvo em', MODEL_PATH)

Modelo salvo em ..\model.joblib
